In [1]:
import numpy as np
import pandas as pd
import glob
import os, sys
import matplotlib.pyplot as plt
import time
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
VERSION = 4
RANDOM_SEED = 26

## Load data

### Load Data - downsampled

In [2]:
# df_joint_train_org   = pd.read_csv(f'./features/cache_all_features_train_V{VERSION}.csv')
# df_joint_train_org   = df_joint_train_org.drop(columns=['GNE_max_gne','GNE_mean_gne','GNE_stddev_gne','GNE_sum_gne'])
                       
# df_joint_test_org = pd.read_csv(f'./features/cache_all_features_test_V{VERSION}.csv').drop(
#     columns=['GNE_max_gne','GNE_mean_gne','GNE_stddev_gne','GNE_sum_gne'])

# print("shape of train set: ", df_joint_train_org.shape)
# print("shape of test  set: ", df_joint_test_org.shape)


# df_joint_train  = pd.read_csv(f'./features/cache_train_V4_resampled_2500.csv')
df_joint_test = pd.read_csv(f'./cache_all_features_test_V4.csv')

# print("shape of train set: ", df_joint_train.shape)
print("shape of test  set: ", df_joint_test.shape)

df_joint_train_aug  = pd.read_csv(f'./cache_train_V4_augmented.csv')
feature_column_names = [i for i in df_joint_train_aug.columns \
                        if i not in ['file_path','renamed_file_path','split','sentiment_value','emotional_category']]
             
print("shape of train set: ", df_joint_train_aug.shape)
df_joint_train_aug.groupby('sentiment_value')['file_path'].count()

shape of test  set:  (1180, 1550)
shape of train set:  (24885, 1546)


sentiment_value
-1    7999
 0    8560
 1    8326
Name: file_path, dtype: int64

### best guess feature combinations

In [3]:
# generate selected features 
def generate_selected_features_by_type(feature_column_names,input,stats,number=1):
    selected_result = []
    for name in feature_column_names:
        if input+"_"+stats in name:
            selected_result.append(name)
    if number < len(selected_result):
        selected_result = selected_result[:number]
    return selected_result

# example to take mfcc 20 mean & std; mel32; zcr all 5 stats features
feature_MFCC20_mean  = generate_selected_features_by_type(feature_column_names,"mfcc","mean",20)
feature_MFCC20_std   = generate_selected_features_by_type(feature_column_names,"mfcc","std",20)
feature_mel32_median = generate_selected_features_by_type(feature_column_names,"mel32","median",32)
feature_mel32_std    = generate_selected_features_by_type(feature_column_names,"mel32","std",32)
feature_zcr_stats    = generate_selected_features_by_type(feature_column_names,"zcr","",5)
feature_rms_stats    = generate_selected_features_by_type(feature_column_names,"rms","",5)
selected_spect = ['Spectrum_band_energy_difference','Spectrum_band_density_difference','Spectrum_center_of_gravity_spectrum','Spectrum_skewness_spectrum','Spectrum_kurtosis_spectrum', 'Spectrum_stddev_spectrum','Spectrum_band_density', 'Spectrum_band_energy']
selected_formant = ['Formant_f1_mean','Formant_f1_median','Formant_f3_mean','Formant_fitch_vtl','Formant_mff','Formant_formant_dispersion']
selected_pitch = ['Pitch_pitch_slope_without_octave_jumps', 'Pitch_q3_pitch','Pitch_stddev_pitch', 'Pitch_mean_absolute_pitch_slope','Pitch_mean_pitch', 'Pitch_max_pitch', 'Pitch_q1_pitch', 'Pitch_min_pitch']
selected_intensity = ['Intensity_max_intensity', 'Intensity_q3_intensity','Intensity_median_intensity', 'Intensity_mean_intensity', 'Intensity_stddev_intensity','Intensity_relative_max_intensity_time']
selected_HNR = ['HNR_stddev_hnr', 'HNR_mean_hnr','HNR_relative_min_hnr_time','HNR_max_hnr']
selected_prosody = selected_intensity + selected_pitch # + ['Local Jitter','Local Shimmer']
selected_feature_names = feature_MFCC20_mean + feature_MFCC20_std + feature_mel32_median + feature_mel32_std + \
                        feature_zcr_stats + feature_rms_stats + selected_intensity + selected_pitch 

In [4]:
# default use augmented training set and balanced test set
X_train = df_joint_train_aug[selected_feature_names]
y_train_s = df_joint_train_aug['sentiment_value']
y_train_e = df_joint_train_aug['emotional_category']

# from sklearn.preprocessing import LabelEncoder
# label_encoder = LabelEncoder()
# y_e_num = label_encoder.fit_transform(y_train_e)

X_test = df_joint_test[selected_feature_names]
y_test_s = df_joint_test['sentiment_value']
y_test_e = df_joint_test['emotional_category']

# y_test_e_num = label_encoder.fit_transform(y_test_e)

In [5]:
X_train.shape, X_test.shape

((24885, 128), (1180, 128))

In [6]:
# !pip install lightgbm

## Models

In [7]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix,accuracy_score, precision_recall_fscore_support
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier,HistGradientBoostingClassifier
from lightgbm import LGBMClassifier

# Common adjustable parameters
common_params = {
    'RandomForest': {'n_estimators': 100, 'criterion':'gini', 'max_depth': None, 
                     'min_samples_split':100, 'bootstrap':True, 'n_jobs':3, 'random_state': RANDOM_SEED},
    'RandomForest1': {'n_estimators': 500, 'criterion':'gini', 'max_depth': None, 
                     'min_samples_split':100, 'bootstrap':True, 'n_jobs':3, 'random_state': RANDOM_SEED},
    'RandomForest2': {'n_estimators': 1000, 'criterion':'gini', 'max_depth': None, 
                     'min_samples_split':100, 'bootstrap':True, 'n_jobs':3, 'random_state': RANDOM_SEED},
    'RandomForest3': {'n_estimators': 100, 'criterion':'gini', 'max_depth': 3, 
                     'min_samples_split':100, 'bootstrap':True, 'n_jobs':3, 'random_state': RANDOM_SEED},
    'RandomForest4': {'n_estimators': 100, 'criterion':'gini', 'max_depth': 5, 
                     'min_samples_split':100, 'bootstrap':True, 'n_jobs':3, 'random_state': RANDOM_SEED},
    'RandomForest5': {'n_estimators': 100, 'criterion':'gini', 'max_depth': 10, 
                     'min_samples_split':100, 'bootstrap':True, 'n_jobs':3, 'random_state': RANDOM_SEED},
    'RandomForest6': {'n_estimators': 100, 'criterion':'gini', 'max_depth': 20, 
                     'min_samples_split':100, 'bootstrap':True, 'n_jobs':3, 'random_state': RANDOM_SEED},
    'RandomForest7': {'n_estimators': 100, 'criterion':'gini', 'max_depth': None, 
                     'min_samples_split':50, 'bootstrap':True, 'n_jobs':3, 'random_state': RANDOM_SEED},
    'RandomForest8': {'n_estimators': 200, 'criterion':'gini', 'max_depth': None, 
                     'min_samples_split':200, 'bootstrap':True, 'n_jobs':3, 'random_state': RANDOM_SEED},
    'RandomForest9': {'n_estimators': 100, 'criterion':'gini', 'max_depth': None, 
                     'min_samples_split':500, 'bootstrap':True, 'n_jobs':3, 'random_state': RANDOM_SEED},
    'RandomForest10': {'n_estimators': 100, 'criterion':'gini', 'max_depth': None, 
                     'min_samples_split':100, 'bootstrap':True, 'n_jobs':5, 'random_state': RANDOM_SEED},
    'RandomForest11': {'n_estimators': 100, 'criterion':'gini', 'max_depth': None, 
                     'min_samples_split':100, 'bootstrap':True, 'n_jobs':10, 'random_state': RANDOM_SEED},
    'RandomForest12': {'n_estimators': 100, 'criterion':'gini', 'max_depth': None, 
                     'min_samples_split':100, 'bootstrap':False, 'n_jobs':3, 'random_state': RANDOM_SEED},

    
    
    'SVM': {'kernel': 'rbf', 'C': 1.0, 'probability': True},
    'KNN': {'n_neighbors': 2},
    'KNN1': {'n_neighbors': 3},
    'KNN2': {'n_neighbors': 4},
    'KNN3': {'n_neighbors': 5},
    'KNN4': {'n_neighbors': 6},
    'KNN5': {'n_neighbors': 8},
    'KNN6': {'n_neighbors': 10},
    'KNN7': {'n_neighbors': 12},
    'KNN8': {'n_neighbors': 15},
    'KNN9': {'n_neighbors': 20},
    'KNN10': {'n_neighbors': 30},
    'GradientBoosting': {'loss': 'log_loss', 'learning_rate': 0.1, 'n_estimators': 100, 'subsample': 1.0,
                         'criterion': 'friedman_mse', 'min_samples_split': 2, 'max_depth': 3},
    'GradientBoostingFast': {'loss': 'log_loss', 'learning_rate': 0.1, 'max_iter': 50},
    'GradientBoostingFast1': {'loss': 'log_loss', 'learning_rate': 0.1, 'max_iter': 100},
    'GradientBoostingFast2': {'loss': 'log_loss', 'learning_rate': 0.1, 'max_iter': 200},
    'GradientBoostingFast3': {'loss': 'log_loss', 'learning_rate': 0.05, 'max_iter': 50},
    'GradientBoostingFast4': {'loss': 'log_loss', 'learning_rate': 0.05, 'max_iter': 100},
    'GradientBoostingFast5': {'loss': 'log_loss', 'learning_rate': 0.05, 'max_iter': 200},
    'GradientBoostingFast6': {'loss': 'log_loss', 'learning_rate': 0.5, 'max_iter': 50},
    'GradientBoostingFast7': {'loss': 'log_loss', 'learning_rate': 0.5, 'max_iter': 100},
    'GradientBoostingFast8': {'loss': 'log_loss', 'learning_rate': 0.5, 'max_iter': 200},
    'GradientBoostingFast9': {'loss': 'log_loss', 'learning_rate': 0.01, 'max_iter': 50},
    'GradientBoostingFast10': {'loss': 'log_loss', 'learning_rate': 0.01, 'max_iter': 100},
    'GradientBoostingFast11': {'loss': 'log_loss', 'learning_rate': 0.01, 'max_iter': 200},
    'GradientBoostingFast12': {'loss': 'log_loss', 'learning_rate': 1, 'max_iter': 30},
    'GradientBoostingFast13': {'loss': 'log_loss', 'learning_rate': 1, 'max_iter': 50},
    'GradientBoostingFast14': {'loss': 'log_loss', 'learning_rate': 1, 'max_iter': 100},
    'AdaBoost': {'n_estimators': 10, 'learning_rate': 1.0},
    'AdaBoost1': {'n_estimators': 20, 'learning_rate': 1.0},
    'AdaBoost2': {'n_estimators': 50, 'learning_rate': 1.0},
    'AdaBoost3': {'n_estimators': 100, 'learning_rate': 1.0},
    'AdaBoost4': {'n_estimators': 50, 'learning_rate': 0.01},
    'AdaBoost5': {'n_estimators': 50, 'learning_rate': 0.05},
    'AdaBoost6': {'n_estimators': 50, 'learning_rate': 0.1},
    'AdaBoost7': {'n_estimators': 50, 'learning_rate': 0.5},
    'AdaBoost8': {'n_estimators': 50, 'learning_rate': 2.0},
    'AdaBoost9': {'n_estimators': 100, 'learning_rate': 0.01},
    'AdaBoost10': {'n_estimators': 100, 'learning_rate': 0.05},
    'AdaBoost11': {'n_estimators': 100, 'learning_rate': 0.1},
    'AdaBoost12': {'n_estimators': 100, 'learning_rate': 0.5},
    'AdaBoost13': {'n_estimators': 100, 'learning_rate': 2.0},
    'AdaBoost14': {'n_estimators': 500, 'learning_rate': 1.0},
    'LightGBM': {'boosting_type': 'gbdt', 'learning_rate': 0.1, 'n_estimators': 100, 'subsample': 1.0,
                 'min_child_samples': 20, 'max_depth': -1}    
}

# Models with common adjustable parameters
dtree   = DecisionTreeClassifier()
rforest  = RandomForestClassifier(**common_params['RandomForest'])
rforest1 = RandomForestClassifier(**common_params['RandomForest1'])
rforest2 = RandomForestClassifier(**common_params['RandomForest2'])
rforest3 = RandomForestClassifier(**common_params['RandomForest3'])
rforest4 = RandomForestClassifier(**common_params['RandomForest4'])
rforest5 = RandomForestClassifier(**common_params['RandomForest5'])
rforest6 = RandomForestClassifier(**common_params['RandomForest6'])
rforest7 = RandomForestClassifier(**common_params['RandomForest7'])
rforest8 = RandomForestClassifier(**common_params['RandomForest8'])
rforest9 = RandomForestClassifier(**common_params['RandomForest9'])
rforest10 = RandomForestClassifier(**common_params['RandomForest10'])
rforest11 = RandomForestClassifier(**common_params['RandomForest11'])
rforest12 = RandomForestClassifier(**common_params['RandomForest12'])

svm     = SVC(**common_params['SVM'])
knn     = KNeighborsClassifier(**common_params['KNN'])
knn1     = KNeighborsClassifier(**common_params['KNN1'])
knn2     = KNeighborsClassifier(**common_params['KNN2'])
knn3     = KNeighborsClassifier(**common_params['KNN3'])
knn4     = KNeighborsClassifier(**common_params['KNN4'])
knn5     = KNeighborsClassifier(**common_params['KNN5'])
knn6     = KNeighborsClassifier(**common_params['KNN6'])
knn7     = KNeighborsClassifier(**common_params['KNN7'])
knn8     = KNeighborsClassifier(**common_params['KNN8'])
knn9     = KNeighborsClassifier(**common_params['KNN9'])
knn10     = KNeighborsClassifier(**common_params['KNN10'])
gboost  = GradientBoostingClassifier(**common_params['GradientBoosting'])
gb_fast = HistGradientBoostingClassifier(**common_params['GradientBoostingFast'])
gb_fast1 = HistGradientBoostingClassifier(**common_params['GradientBoostingFast1'])
gb_fast2 = HistGradientBoostingClassifier(**common_params['GradientBoostingFast2'])
gb_fast3 = HistGradientBoostingClassifier(**common_params['GradientBoostingFast3'])
gb_fast4 = HistGradientBoostingClassifier(**common_params['GradientBoostingFast4'])
gb_fast5 = HistGradientBoostingClassifier(**common_params['GradientBoostingFast5'])
gb_fast6 = HistGradientBoostingClassifier(**common_params['GradientBoostingFast6'])
gb_fast7 = HistGradientBoostingClassifier(**common_params['GradientBoostingFast7'])
gb_fast8 = HistGradientBoostingClassifier(**common_params['GradientBoostingFast8'])
gb_fast9 = HistGradientBoostingClassifier(**common_params['GradientBoostingFast9'])
gb_fast10 = HistGradientBoostingClassifier(**common_params['GradientBoostingFast10'])
gb_fast11 = HistGradientBoostingClassifier(**common_params['GradientBoostingFast11'])
gb_fast12 = HistGradientBoostingClassifier(**common_params['GradientBoostingFast12'])
gb_fast13 = HistGradientBoostingClassifier(**common_params['GradientBoostingFast13'])
gb_fast14 = HistGradientBoostingClassifier(**common_params['GradientBoostingFast14'])
adaBoost= AdaBoostClassifier(**common_params['AdaBoost'])
adaBoost1= AdaBoostClassifier(**common_params['AdaBoost1'])
adaBoost2= AdaBoostClassifier(**common_params['AdaBoost2'])
adaBoost3= AdaBoostClassifier(**common_params['AdaBoost3'])
adaBoost4= AdaBoostClassifier(**common_params['AdaBoost4'])
adaBoost5= AdaBoostClassifier(**common_params['AdaBoost5'])
adaBoost6= AdaBoostClassifier(**common_params['AdaBoost6'])
adaBoost7= AdaBoostClassifier(**common_params['AdaBoost7'])
adaBoost8= AdaBoostClassifier(**common_params['AdaBoost8'])
adaBoost9= AdaBoostClassifier(**common_params['AdaBoost9'])
adaBoost10= AdaBoostClassifier(**common_params['AdaBoost10'])
adaBoost11= AdaBoostClassifier(**common_params['AdaBoost11'])
adaBoost12= AdaBoostClassifier(**common_params['AdaBoost12'])
adaBoost13= AdaBoostClassifier(**common_params['AdaBoost13'])
adaBoost14= AdaBoostClassifier(**common_params['AdaBoost14'])
lightgbm=LGBMClassifier(**common_params['LightGBM'])

In [8]:
# def exp_clf_with_feature_selected(clf_model, X_train, X_test, y_train, y_test):
#     start = time.time()
#     print(f'Model Name: {clf_model.__class__};\n Train set shape {X_train.shape}, num of class {y_train.unique().size}')
#     predictions = clf_model.fit(X_train, y_train).predict(X_test.values)
    
#     print(classification_report(y_test, predictions))
#     print(confusion_matrix(y_test, predictions))
    
#     precision, recall, f1score, support = precision_recall_fscore_support(y_test, predictions, average=None)
#     # TODO make all metrics into result dict for recording 
#     probabilities = clf_model.predict_proba(X_test.values)
#     print('prbabilities distribution: \n', pd.DataFrame(probabilities,columns=clf_model.classes_).describe())
#     print(f'Time taken: {round(time.time()-start,3)} seconds.\n')

In [9]:
import pickle

def exp_clf_with_feature_selected(clf_model, X_train, X_test, y_train, y_test,verbose=True):
    start = time.time()
    
    clf_model.fit(X_train, y_train)
    predictions = clf_model.predict(X_test.values)
    
    # Calculate metrics
    report = classification_report(y_test, predictions, output_dict=True)
    metrics = {
        'accuracy': report['accuracy'],
        'precision': report['macro avg']['precision'],
        'recall': report['macro avg']['recall'],
        'f1-score': report['macro avg']['f1-score']
    }
    for class_name in report.keys():
        if class_name not in ['accuracy', 'macro avg', 'weighted avg']:
            metrics[class_name+'_precision'] = report[class_name]['precision']
            metrics[class_name+'_recall'] = report[class_name]['recall'],
            metrics[class_name+'_f1-score'] = report[class_name]['f1-score']
    
    feature_columns = list(X_train.columns)
    num_classes = y_train.nunique()
    class_names = list(y_train.unique())
    
    model_filename = f"./models/{clf_model.__class__.__name__}_model"
    model_filename += f"_{num_classes}cls_{len(feature_columns)}feat_{round(report['accuracy']*100)}acc.pkl"
    with open(model_filename, 'wb') as file:
        pickle.dump(clf_model, file)
    
    results = {**metrics,
        'num_classes': num_classes,
        'class_names': class_names,
        'model_filename': model_filename,
        'feature_columns': feature_columns,
    }
    
    if verbose:
        print(f"Model Name: {clf_model.__class__.__name__};\nTrain set shape {X_train.shape}, num of class {num_classes}")
        print(classification_report(y_test, predictions))
        print(confusion_matrix(y_test, predictions))
        probabilities = clf_model.predict_proba(X_test.values)
        print('Probabilities distribution:\n', pd.DataFrame(probabilities, columns=clf_model.classes_).describe())
    print(f"Model: {clf_model.__class__.__name__};Time taken: {round(time.time()-start, 3)} seconds.\n")

    return results, clf_model


### Sentiment 3-class Classifier Sample code

In [10]:
result, m_trained = exp_clf_with_feature_selected(adaBoost, X_train, X_test, y_train_s, y_test_s)

Model Name: AdaBoostClassifier;
Train set shape (24885, 128), num of class 3
              precision    recall  f1-score   support

          -1       0.82      0.36      0.50       748
           0       0.41      0.82      0.54       183
           1       0.38      0.73      0.50       249

    accuracy                           0.51      1180
   macro avg       0.54      0.64      0.51      1180
weighted avg       0.67      0.51      0.51      1180

[[268 192 288]
 [ 19 150  14]
 [ 38  28 183]]
Probabilities distribution:
                 -1            0            1
count  1180.000000  1180.000000  1180.000000
mean      0.352335     0.302797     0.344868
std       0.029347     0.070906     0.048899
min       0.298309     0.152966     0.260133
25%       0.331102     0.244784     0.300263
50%       0.339916     0.326899     0.336636
75%       0.368369     0.354011     0.389033
max       0.423467     0.441557     0.431561
Model: AdaBoostClassifier;Time taken: 18.11 seconds.



In [11]:
result, m_trained = exp_clf_with_feature_selected(adaBoost1, X_train, X_test, y_train_s, y_test_s)

Model Name: AdaBoostClassifier;
Train set shape (24885, 128), num of class 3
              precision    recall  f1-score   support

          -1       0.82      0.54      0.65       748
           0       0.43      0.81      0.56       183
           1       0.45      0.63      0.53       249

    accuracy                           0.60      1180
   macro avg       0.57      0.66      0.58      1180
weighted avg       0.68      0.60      0.61      1180

[[401 165 182]
 [ 27 148   8]
 [ 63  29 157]]
Probabilities distribution:
                 -1            0            1
count  1180.000000  1180.000000  1180.000000
mean      0.350967     0.307959     0.341075
std       0.020536     0.047423     0.031902
min       0.308292     0.186604     0.281038
25%       0.335552     0.274330     0.315570
50%       0.345877     0.324330     0.333946
75%       0.362773     0.343312     0.365815
max       0.418306     0.400672     0.426674
Model: AdaBoostClassifier;Time taken: 38.96 seconds.



In [12]:
result, m_trained = exp_clf_with_feature_selected(adaBoost2, X_train, X_test, y_train_s, y_test_s)

Model Name: AdaBoostClassifier;
Train set shape (24885, 128), num of class 3
              precision    recall  f1-score   support

          -1       0.77      0.58      0.66       748
           0       0.45      0.71      0.55       183
           1       0.46      0.61      0.52       249

    accuracy                           0.61      1180
   macro avg       0.56      0.63      0.58      1180
weighted avg       0.65      0.61      0.61      1180

[[432 141 175]
 [ 48 130   5]
 [ 81  16 152]]
Probabilities distribution:
                 -1            0            1
count  1180.000000  1180.000000  1180.000000
mean      0.344585     0.317610     0.337805
std       0.012496     0.028981     0.018914
min       0.320979     0.168548     0.294940
25%       0.336330     0.303274     0.323921
50%       0.341487     0.327776     0.333314
75%       0.349684     0.337500     0.348896
max       0.416578     0.373587     0.426871
Model: AdaBoostClassifier;Time taken: 98.79 seconds.



In [13]:
result, m_trained = exp_clf_with_feature_selected(adaBoost3, X_train, X_test, y_train_s, y_test_s)

Model Name: AdaBoostClassifier;
Train set shape (24885, 128), num of class 3
              precision    recall  f1-score   support

          -1       0.76      0.62      0.68       748
           0       0.49      0.74      0.59       183
           1       0.46      0.54      0.50       249

    accuracy                           0.62      1180
   macro avg       0.57      0.63      0.59      1180
weighted avg       0.65      0.62      0.63      1180

[[462 129 157]
 [ 44 136   3]
 [101  13 135]]
Probabilities distribution:
                 -1            0            1
count  1180.000000  1180.000000  1180.000000
mean      0.341720     0.321530     0.336750
std       0.009592     0.021360     0.013600
min       0.323871     0.221025     0.253976
25%       0.335312     0.311988     0.327826
50%       0.338846     0.330755     0.332856
75%       0.344920     0.335480     0.344545
max       0.389491     0.370691     0.392303
Model: AdaBoostClassifier;Time taken: 192.785 seconds.



In [14]:
result, m_trained = exp_clf_with_feature_selected(adaBoost4, X_train, X_test, y_train_s, y_test_s)

Model Name: AdaBoostClassifier;
Train set shape (24885, 128), num of class 3
              precision    recall  f1-score   support

          -1       0.76      0.03      0.06       748
           0       0.24      0.97      0.38       183
           1       0.36      0.57      0.44       249

    accuracy                           0.29      1180
   macro avg       0.45      0.52      0.29      1180
weighted avg       0.59      0.29      0.19      1180

[[ 25 474 249]
 [  2 177   4]
 [  6 101 142]]
Probabilities distribution:
                 -1            0            1
count  1180.000000  1180.000000  1180.000000
mean      0.337522     0.323866     0.338613
std       0.012317     0.169979     0.166794
min       0.331048     0.067982     0.214628
25%       0.331048     0.090412     0.214628
50%       0.333615     0.446614     0.218951
75%       0.334435     0.454324     0.561288
max       0.382076     0.454324     0.598404
Model: AdaBoostClassifier;Time taken: 101.886 seconds.



In [15]:
result, m_trained = exp_clf_with_feature_selected(adaBoost5, X_train, X_test, y_train_s, y_test_s)

Model Name: AdaBoostClassifier;
Train set shape (24885, 128), num of class 3
              precision    recall  f1-score   support

          -1       0.68      0.22      0.34       748
           0       0.28      0.84      0.42       183
           1       0.37      0.59      0.46       249

    accuracy                           0.40      1180
   macro avg       0.44      0.55      0.41      1180
weighted avg       0.55      0.40      0.37      1180

[[167 338 243]
 [ 27 154   2]
 [ 53  50 146]]
Probabilities distribution:
                 -1            0            1
count  1180.000000  1180.000000  1180.000000
mean      0.355688     0.304603     0.339709
std       0.017576     0.100595     0.086424
min       0.311641     0.127089     0.223234
25%       0.341635     0.200388     0.265612
50%       0.353689     0.340909     0.305825
75%       0.370282     0.393532     0.422196
max       0.392330     0.465125     0.510227
Model: AdaBoostClassifier;Time taken: 102.457 seconds.



In [16]:
result, m_trained = exp_clf_with_feature_selected(adaBoost6, X_train, X_test, y_train_s, y_test_s)

Model Name: AdaBoostClassifier;
Train set shape (24885, 128), num of class 3
              precision    recall  f1-score   support

          -1       0.71      0.27      0.40       748
           0       0.32      0.84      0.46       183
           1       0.38      0.62      0.47       249

    accuracy                           0.43      1180
   macro avg       0.47      0.58      0.44      1180
weighted avg       0.58      0.43      0.42      1180

[[205 291 252]
 [ 27 153   3]
 [ 56  38 155]]
Probabilities distribution:
                 -1            0            1
count  1180.000000  1180.000000  1180.000000
mean      0.354901     0.303730     0.341369
std       0.019200     0.076621     0.062056
min       0.301869     0.151190     0.239941
25%       0.339352     0.239863     0.289652
50%       0.349159     0.332062     0.322153
75%       0.368485     0.367701     0.391728
max       0.400763     0.455698     0.473654
Model: AdaBoostClassifier;Time taken: 96.403 seconds.



In [17]:
result, m_trained = exp_clf_with_feature_selected(adaBoost7, X_train, X_test, y_train_s, y_test_s)

Model Name: AdaBoostClassifier;
Train set shape (24885, 128), num of class 3
              precision    recall  f1-score   support

          -1       0.79      0.49      0.60       748
           0       0.43      0.78      0.55       183
           1       0.43      0.67      0.53       249

    accuracy                           0.57      1180
   macro avg       0.55      0.65      0.56      1180
weighted avg       0.66      0.57      0.58      1180

[[364 170 214]
 [ 35 143   5]
 [ 61  21 167]]
Probabilities distribution:
                 -1            0            1
count  1180.000000  1180.000000  1180.000000
mean      0.347124     0.314107     0.338769
std       0.014167     0.036143     0.025800
min       0.313023     0.212029     0.280785
25%       0.336762     0.288930     0.319069
50%       0.343373     0.328056     0.331877
75%       0.355256     0.341934     0.356102
max       0.401363     0.391595     0.412511
Model: AdaBoostClassifier;Time taken: 97.534 seconds.



In [18]:
result, m_trained = exp_clf_with_feature_selected(adaBoost8, X_train, X_test, y_train_s, y_test_s)

Model Name: AdaBoostClassifier;
Train set shape (24885, 128), num of class 3
              precision    recall  f1-score   support

          -1       0.64      0.67      0.65       748
           0       0.01      0.02      0.01       183
           1       0.31      0.02      0.03       249

    accuracy                           0.43      1180
   macro avg       0.32      0.23      0.23      1180
weighted avg       0.47      0.43      0.42      1180

[[498 243   7]
 [178   3   2]
 [108 137   4]]
Probabilities distribution:
                 -1            0            1
count  1180.000000  1180.000000  1180.000000
mean      0.333351     0.333211     0.333437
std       0.000224     0.001307     0.001086
min       0.333273     0.320456     0.333314
25%       0.333273     0.333318     0.333314
50%       0.333355     0.333318     0.333327
75%       0.333355     0.333414     0.333327
max       0.335614     0.333414     0.343930
Model: AdaBoostClassifier;Time taken: 97.225 seconds.



In [19]:
result, m_trained = exp_clf_with_feature_selected(adaBoost9, X_train, X_test, y_train_s, y_test_s)

Model Name: AdaBoostClassifier;
Train set shape (24885, 128), num of class 3
              precision    recall  f1-score   support

          -1       0.67      0.11      0.18       748
           0       0.25      0.93      0.40       183
           1       0.36      0.56      0.44       249

    accuracy                           0.33      1180
   macro avg       0.43      0.53      0.34      1180
weighted avg       0.54      0.33      0.27      1180

[[ 80 422 246]
 [  8 170   5]
 [ 32  77 140]]
Probabilities distribution:
                 -1            0            1
count  1180.000000  1180.000000  1180.000000
mean      0.349195     0.312657     0.338148
std       0.015041     0.137002     0.127076
min       0.327808     0.096433     0.220849
25%       0.334283     0.149412     0.232575
50%       0.349922     0.376100     0.265565
75%       0.360120     0.432842     0.484590
max       0.384031     0.451344     0.553645
Model: AdaBoostClassifier;Time taken: 196.835 seconds.



In [20]:
result, m_trained = exp_clf_with_feature_selected(adaBoost10, X_train, X_test, y_train_s, y_test_s)

Model Name: AdaBoostClassifier;
Train set shape (24885, 128), num of class 3
              precision    recall  f1-score   support

          -1       0.70      0.25      0.37       748
           0       0.31      0.85      0.45       183
           1       0.38      0.62      0.47       249

    accuracy                           0.42      1180
   macro avg       0.46      0.57      0.43      1180
weighted avg       0.57      0.42      0.40      1180

[[188 308 252]
 [ 25 155   3]
 [ 56  39 154]]
Probabilities distribution:
                 -1            0            1
count  1180.000000  1180.000000  1180.000000
mean      0.354402     0.303926     0.341672
std       0.018714     0.076494     0.061863
min       0.301436     0.149079     0.239541
25%       0.339461     0.240728     0.289944
50%       0.349224     0.331757     0.321952
75%       0.368205     0.367137     0.392406
max       0.397398     0.457776     0.475972
Model: AdaBoostClassifier;Time taken: 196.579 seconds.



In [21]:
result, m_trained = exp_clf_with_feature_selected(adaBoost11, X_train, X_test, y_train_s, y_test_s)

Model Name: AdaBoostClassifier;
Train set shape (24885, 128), num of class 3
              precision    recall  f1-score   support

          -1       0.80      0.36      0.50       748
           0       0.37      0.89      0.52       183
           1       0.40      0.65      0.50       249

    accuracy                           0.51      1180
   macro avg       0.52      0.63      0.51      1180
weighted avg       0.65      0.51      0.50      1180

[[273 240 235]
 [ 18 162   3]
 [ 52  36 161]]
Probabilities distribution:
                 -1            0            1
count  1180.000000  1180.000000  1180.000000
mean      0.351589     0.307480     0.340932
std       0.018594     0.057128     0.042664
min       0.309990     0.172245     0.263470
25%       0.337357     0.265821     0.306726
50%       0.344811     0.330168     0.328451
75%       0.362823     0.352535     0.373903
max       0.407303     0.423470     0.444758
Model: AdaBoostClassifier;Time taken: 198.47 seconds.



In [22]:
result, m_trained = exp_clf_with_feature_selected(adaBoost12, X_train, X_test, y_train_s, y_test_s)

Model Name: AdaBoostClassifier;
Train set shape (24885, 128), num of class 3
              precision    recall  f1-score   support

          -1       0.78      0.55      0.64       748
           0       0.48      0.78      0.59       183
           1       0.43      0.61      0.50       249

    accuracy                           0.60      1180
   macro avg       0.56      0.64      0.58      1180
weighted avg       0.66      0.60      0.60      1180

[[409 140 199]
 [ 35 143   5]
 [ 80  18 151]]
Probabilities distribution:
                 -1            0            1
count  1180.000000  1180.000000  1180.000000
mean      0.354925     0.295922     0.349153
std       0.030532     0.067574     0.038205
min       0.320146     0.108417     0.244965
25%       0.335845     0.296765     0.324938
50%       0.340518     0.329266     0.332382
75%       0.362976     0.337866     0.358300
max       0.451526     0.375109     0.459522
Model: AdaBoostClassifier;Time taken: 198.729 seconds.



In [23]:
result, m_trained = exp_clf_with_feature_selected(adaBoost13, X_train, X_test, y_train_s, y_test_s)

Model Name: AdaBoostClassifier;
Train set shape (24885, 128), num of class 3
              precision    recall  f1-score   support

          -1       0.63      0.67      0.65       748
           0       0.01      0.02      0.01       183
           1       0.22      0.02      0.04       249

    accuracy                           0.43      1180
   macro avg       0.29      0.23      0.23      1180
weighted avg       0.45      0.43      0.42      1180

[[499 233  16]
 [178   3   2]
 [110 134   5]]
Probabilities distribution:
                 -1            0            1
count  1180.000000  1180.000000  1180.000000
mean      0.333349     0.333230     0.333421
std       0.000171     0.000904     0.000738
min       0.333255     0.326471     0.333297
25%       0.333255     0.333315     0.333297
50%       0.333358     0.333315     0.333326
75%       0.333358     0.333448     0.333326
max       0.334642     0.333448     0.338887
Model: AdaBoostClassifier;Time taken: 188.755 seconds.



In [24]:
result, m_trained = exp_clf_with_feature_selected(adaBoost14, X_train, X_test, y_train_s, y_test_s)

Model Name: AdaBoostClassifier;
Train set shape (24885, 128), num of class 3
              precision    recall  f1-score   support

          -1       0.75      0.64      0.69       748
           0       0.56      0.66      0.60       183
           1       0.40      0.52      0.45       249

    accuracy                           0.62      1180
   macro avg       0.57      0.60      0.58      1180
weighted avg       0.64      0.62      0.63      1180

[[478  82 188]
 [ 54 120   9]
 [108  12 129]]
Probabilities distribution:
                 -1            0            1
count  1180.000000  1180.000000  1180.000000
mean      0.338375     0.325148     0.336477
std       0.006811     0.014952     0.008665
min       0.329096     0.268152     0.310257
25%       0.334143     0.324987     0.331559
50%       0.335223     0.332288     0.333177
75%       0.339230     0.333883     0.338159
max       0.365193     0.345959     0.369330
Model: AdaBoostClassifier;Time taken: 954.614 seconds.



In [25]:
# change model as the first parameter in the function 
# result, m_trained = exp_clf_with_feature_selected(gb_fast, X_train, X_test, y_train_s, y_test_s)

In [26]:
# result, m_trained = exp_clf_with_feature_selected(adaBoost, X_train, X_test, y_train_s, y_test_s)

In [27]:
# result, m_trained = exp_clf_with_feature_selected(gboost, X_train, X_test, y_train_s, y_test_s)

In [28]:
# result, m_trained = exp_clf_with_feature_selected(knn, X_train, X_test, y_train_s, y_test_s)

In [29]:
# result, m_trained = exp_clf_with_feature_selected(svm, X_train, X_test, y_train_s, y_test_s)

In [30]:
# result, m_trained = exp_clf_with_feature_selected(rforest, X_train, X_test, y_train_s, y_test_s)

In [31]:
# result, m_trained = exp_clf_with_feature_selected(lightgbm, X_train, X_test, y_train_s, y_test_s)

In [32]:
# result, m_trained = exp_clf_with_feature_selected(dtree, X_train, X_test, y_train_s, y_test_s)

### How to save experiment metrics result

In [33]:
exp_results = []
for clf_model in [adaBoost,adaBoost1,adaBoost2,adaBoost3,adaBoost4,adaBoost5,adaBoost6,adaBoost7,adaBoost8,adaBoost9,adaBoost10,adaBoost11,adaBoost12,adaBoost13,adaBoost14]:
    result, m_trained = exp_clf_with_feature_selected(clf_model, X_train, X_test, y_train_s, y_test_s,verbose=False)
    exp_results.append(result)
pd.DataFrame(exp_results)

Model: AdaBoostClassifier;Time taken: 20.436 seconds.

Model: AdaBoostClassifier;Time taken: 40.007 seconds.

Model: AdaBoostClassifier;Time taken: 95.152 seconds.

Model: AdaBoostClassifier;Time taken: 188.053 seconds.

Model: AdaBoostClassifier;Time taken: 96.832 seconds.

Model: AdaBoostClassifier;Time taken: 94.123 seconds.

Model: AdaBoostClassifier;Time taken: 95.853 seconds.

Model: AdaBoostClassifier;Time taken: 95.699 seconds.

Model: AdaBoostClassifier;Time taken: 93.365 seconds.

Model: AdaBoostClassifier;Time taken: 195.471 seconds.

Model: AdaBoostClassifier;Time taken: 196.994 seconds.

Model: AdaBoostClassifier;Time taken: 198.568 seconds.

Model: AdaBoostClassifier;Time taken: 196.328 seconds.

Model: AdaBoostClassifier;Time taken: 196.601 seconds.

Model: AdaBoostClassifier;Time taken: 983.872 seconds.



,accuracy,precision,recall,f1-score,-1_precision,-1_recall,-1_f1-score,0_precision,0_recall,0_f1-score,1_precision,1_recall,1_f1-score,num_classes,class_names,model_filename,feature_columns
0,0.509322,0.535780,0.637634,0.513556,0.824615,"(0.3582887700534759,)",0.499534,0.405405,"(0.819672131147541,)",0.542495,0.377320,"(0.7349397590361446,)",0.498638,3,"[-1, 1, 0]",./models/AdaBoostClassifier_model_3cls_128feat...,"[mfcc_mean_1, mfcc_mean_2, mfcc_mean_3, mfcc_m..."
1,0.598305,0.567300,0.658454,0.579317,0.816701,"(0.536096256684492,)",0.647296,0.432749,"(0.8087431693989071,)",0.563810,0.452450,"(0.6305220883534136,)",0.526846,3,"[-1, 1, 0]",./models/AdaBoostClassifier_model_3cls_128feat...,"[mfcc_mean_1, mfcc_mean_2, mfcc_mean_3, mfcc_m..."
2,0.605085,0.560282,0.632788,0.578824,0.770053,"(0.5775401069518716,)",0.660046,0.452962,"(0.7103825136612022,)",0.553191,0.457831,"(0.6104417670682731,)",0.523236,3,"[-1, 1, 0]",./models/AdaBoostClassifier_model_3cls_128feat...,"[mfcc_mean_1, mfcc_mean_2, mfcc_mean_3, mfcc_m..."
3,0.621186,0.569319,0.634328,0.589421,0.761120,"(0.6176470588235294,)",0.681919,0.489209,"(0.7431693989071039,)",0.590022,0.457627,"(0.5421686746987951,)",0.496324,3,"[-1, 1, 0]",./models/AdaBoostClassifier_model_3cls_128feat...,"[mfcc_mean_1, mfcc_mean_2, mfcc_mean_3, mfcc_m..."
4,0.291525,0.450814,0.523639,0.294541,0.757576,"(0.03342245989304813,)",0.064020,0.235372,"(0.9672131147540983,)",0.378610,0.359494,"(0.570281124497992,)",0.440994,3,"[-1, 1, 0]",./models/AdaBoostClassifier_model_3cls_128feat...,"[mfcc_mean_1, mfcc_mean_2, mfcc_mean_3, mfcc_m..."
5,0.395763,0.444549,0.550379,0.405585,0.676113,"(0.2232620320855615,)",0.335678,0.284133,"(0.8415300546448088,)",0.424828,0.373402,"(0.5863453815261044,)",0.456250,3,"[-1, 1, 0]",./models/AdaBoostClassifier_model_3cls_128feat...,"[mfcc_mean_1, mfcc_mean_2, mfcc_mean_3, mfcc_m..."
6,0.434746,0.469094,0.577540,0.442104,0.711806,"(0.27406417112299464,)",0.395753,0.317427,"(0.8360655737704918,)",0.460150,0.378049,"(0.6224899598393574,)",0.470410,3,"[-1, 1, 0]",./models/AdaBoostClassifier_model_3cls_128feat...,"[mfcc_mean_1, mfcc_mean_2, mfcc_mean_3, mfcc_m..."
7,0.571186,0.550697,0.646245,0.560608,0.791304,"(0.48663101604278075,)",0.602649,0.428144,"(0.7814207650273224,)",0.553191,0.432642,"(0.6706827309236948,)",0.525984,3,"[-1, 1, 0]",./models/AdaBoostClassifier_model_3cls_128feat...,"[mfcc_mean_1, mfcc_mean_2, mfcc_mean_3, mfcc_m..."
8,0.427966,0.316910,0.232744,0.230422,0.635204,"(0.6657754010695187,)",0.650131,0.007833,"(0.01639344262295082,)",0.010601,0.307692,"(0.01606425702811245,)",0.030534,3,"[-1, 1, 0]",./models/AdaBoostClassifier_model_3cls_128feat...,"[mfcc_mean_1, mfcc_mean_2, mfcc_mean_3, mfcc_m..."
9,0.330508,0.426278,0.532721,0.340298,0.666667,"(0.10695187165775401,)",0.184332,0.254111,"(0.9289617486338798,)",0.399061,0.358056,"(0.5622489959839357,)",0.437500,3,"[-1, 1, 0]",./models/AdaBoostClassifier_model_3cls_128feat...,"[mfcc_mean_1, mfcc_mean_2, mfcc_mean_3, mfcc_m..."


In [34]:
# exp_results = []
# for clf_model in [rforest,adaBoost,gb_fast,gboost,knn,svm,lightgbm,dtree]:
#     result, m_trained = exp_clf_with_feature_selected(clf_model, X_train, X_test, y_train_s, y_test_s,verbose=False)
#     exp_results.append(result)
# pd.DataFrame(exp_results)

In [35]:
pd.DataFrame(exp_results).to_excel("exp_result-0316-v4-aug-adaBoost.xlsx")

### Emotion 8-class

In [36]:
exp_results = []
for clf_model in [adaBoost,adaBoost1,adaBoost2,adaBoost3,adaBoost4,adaBoost5,adaBoost6,adaBoost7,adaBoost8,adaBoost9,adaBoost10,adaBoost11,adaBoost12,adaBoost13,adaBoost14]:
    result, m_trained = exp_clf_with_feature_selected(clf_model, X_train, X_test, y_train_e, y_test_e,verbose=False)
    exp_results.append(result)
pd.DataFrame(exp_results)

Model: AdaBoostClassifier;Time taken: 21.33 seconds.

Model: AdaBoostClassifier;Time taken: 41.514 seconds.

Model: AdaBoostClassifier;Time taken: 104.314 seconds.

Model: AdaBoostClassifier;Time taken: 209.664 seconds.

Model: AdaBoostClassifier;Time taken: 104.938 seconds.

Model: AdaBoostClassifier;Time taken: 103.909 seconds.

Model: AdaBoostClassifier;Time taken: 105.482 seconds.

Model: AdaBoostClassifier;Time taken: 104.083 seconds.

Model: AdaBoostClassifier;Time taken: 105.138 seconds.

Model: AdaBoostClassifier;Time taken: 208.466 seconds.

Model: AdaBoostClassifier;Time taken: 209.243 seconds.

Model: AdaBoostClassifier;Time taken: 209.527 seconds.

Model: AdaBoostClassifier;Time taken: 211.015 seconds.

Model: AdaBoostClassifier;Time taken: 212.015 seconds.

Model: AdaBoostClassifier;Time taken: 1047.526 seconds.



,accuracy,precision,recall,f1-score,Anger_precision,Anger_recall,Anger_f1-score,Calmness_precision,Calmness_recall,Calmness_f1-score,...,Sadness_precision,Sadness_recall,Sadness_f1-score,Surprise_precision,Surprise_recall,Surprise_f1-score,num_classes,class_names,model_filename,feature_columns
0,0.294068,0.316208,0.408745,0.276272,0.593496,"(0.39037433155080214,)",0.470968,0.378378,"(0.875,)",0.528302,...,0.000000,"(0.0,)",0.000000,0.432432,"(0.7741935483870968,)",0.554913,8,"[Disgust, Anger, Fear, Sadness, Happiness, Sur...",./models/AdaBoostClassifier_model_8cls_128feat...,"[mfcc_mean_1, mfcc_mean_2, mfcc_mean_3, mfcc_m..."
1,0.317797,0.470900,0.433891,0.306367,0.512195,"(0.44919786096256686,)",0.478632,0.444444,"(1.0,)",0.615385,...,1.000000,"(0.0053475935828877,)",0.010638,0.500000,"(0.6935483870967742,)",0.581081,8,"[Disgust, Anger, Fear, Sadness, Happiness, Sur...",./models/AdaBoostClassifier_model_8cls_128feat...,"[mfcc_mean_1, mfcc_mean_2, mfcc_mean_3, mfcc_m..."
2,0.326271,0.363534,0.427640,0.331068,0.417722,"(0.7058823529411765,)",0.524851,0.517241,"(0.9375,)",0.666667,...,0.181818,"(0.0213903743315508,)",0.038278,0.475610,"(0.6290322580645161,)",0.541667,8,"[Disgust, Anger, Fear, Sadness, Happiness, Sur...",./models/AdaBoostClassifier_model_8cls_128feat...,"[mfcc_mean_1, mfcc_mean_2, mfcc_mean_3, mfcc_m..."
3,0.338136,0.401282,0.400673,0.339943,0.363874,"(0.7433155080213903,)",0.488576,0.560000,"(0.875,)",0.682927,...,0.488372,"(0.11229946524064172,)",0.182609,0.367347,"(0.2903225806451613,)",0.324324,8,"[Disgust, Anger, Fear, Sadness, Happiness, Sur...",./models/AdaBoostClassifier_model_8cls_128feat...,"[mfcc_mean_1, mfcc_mean_2, mfcc_mean_3, mfcc_m..."
4,0.216949,0.055168,0.183932,0.082848,0.000000,"(0.0,)",0.000000,0.000000,"(0.0,)",0.000000,...,0.000000,"(0.0,)",0.000000,0.000000,"(0.0,)",0.000000,8,"[Disgust, Anger, Fear, Sadness, Happiness, Sur...",./models/AdaBoostClassifier_model_8cls_128feat...,"[mfcc_mean_1, mfcc_mean_2, mfcc_mean_3, mfcc_m..."
5,0.226271,0.127778,0.196755,0.099841,0.000000,"(0.0,)",0.000000,0.000000,"(0.0,)",0.000000,...,0.000000,"(0.0,)",0.000000,0.571429,"(0.06451612903225806,)",0.115942,8,"[Disgust, Anger, Fear, Sadness, Happiness, Sur...",./models/AdaBoostClassifier_model_8cls_128feat...,"[mfcc_mean_1, mfcc_mean_2, mfcc_mean_3, mfcc_m..."
6,0.243220,0.168073,0.236307,0.147134,0.000000,"(0.0,)",0.000000,0.333333,"(0.1875,)",0.240000,...,0.000000,"(0.0,)",0.000000,0.533333,"(0.12903225806451613,)",0.207792,8,"[Disgust, Anger, Fear, Sadness, Happiness, Sur...",./models/AdaBoostClassifier_model_8cls_128feat...,"[mfcc_mean_1, mfcc_mean_2, mfcc_mean_3, mfcc_m..."
7,0.299153,0.374680,0.331183,0.281568,0.367347,"(0.48128342245989303,)",0.416667,0.444444,"(0.5,)",0.470588,...,0.500000,"(0.026737967914438502,)",0.050761,0.446809,"(0.3387096774193548,)",0.385321,8,"[Disgust, Anger, Fear, Sadness, Happiness, Sur...",./models/AdaBoostClassifier_model_8cls_128feat...,"[mfcc_mean_1, mfcc_mean_2, mfcc_mean_3, mfcc_m..."
8,0.222881,0.185284,0.304171,0.174270,0.238683,"(0.93048128342246,)",0.379913,0.333333,"(1.0,)",0.500000,...,0.370968,"(0.12299465240641712,)",0.184739,0.142857,"(0.12903225806451613,)",0.135593,8,"[Disgust, Anger, Fear, Sadness, Happiness, Sur...",./models/AdaBoostClassifier_model_8cls_128feat...,"[mfcc_mean_1, mfcc_mean_2, mfcc_mean_3, mfcc_m..."
9,0.221186,0.180903,0.188622,0.088230,0.000000,"(0.0,)",0.000000,0.000000,"(0.0,)",0.000000,...,0.000000,"(0.0,)",0.000000,1.000000,"(0.016129032258064516,)",0.031746,8,"[Disgust, Anger, Fear, Sadness, Happiness, Sur...",./models/AdaBoostClassifier_model_8cls_128feat...,"[mfcc_mean_1, mfcc_mean_2, mfcc_mean_3, mfcc_m..."


In [37]:
pd.DataFrame(exp_results).to_excel("exp_result-0316-v4-aug-adaBoost-8.xlsx")

In [38]:
exp_clf_with_feature_selected(adaBoost, X_train, X_test, y_train_e, y_test_e)

Model Name: AdaBoostClassifier;
Train set shape (24885, 128), num of class 8
              precision    recall  f1-score   support

       Anger       0.59      0.39      0.47       187
    Calmness       0.38      0.88      0.53        16
     Disgust       0.67      0.01      0.02       187
        Fear       0.00      0.00      0.00       187
   Happiness       0.22      0.32      0.26       187
  Neutrality       0.24      0.90      0.37       167
     Sadness       0.00      0.00      0.00       187
    Surprise       0.43      0.77      0.55        62

    accuracy                           0.29      1180
   macro avg       0.32      0.41      0.28      1180
weighted avg       0.30      0.29      0.21      1180

[[ 73   1   1   0  80  27   0   5]
 [  0  14   0   0   0   2   0   0]
 [  6   3   2   0  30 140   0   6]
 [ 22   2   0   0  63  87   0  13]
 [ 22   2   0   0  59  72   0  32]
 [  0   5   0   0  10 151   0   1]
 [  0   7   0   0  14 160   0   6]
 [  0   3   0   0   8   3  

({'accuracy': 0.2940677966101695,
  'precision': 0.3162075941416892,
  'recall': 0.4087453381575645,
  'f1-score': 0.2762720905198649,
  'Anger_precision': 0.5934959349593496,
  'Anger_recall': (0.39037433155080214,),
  'Anger_f1-score': 0.47096774193548385,
  'Calmness_precision': 0.3783783783783784,
  'Calmness_recall': (0.875,),
  'Calmness_f1-score': 0.5283018867924528,
  'Disgust_precision': 0.6666666666666666,
  'Disgust_recall': (0.0106951871657754,),
  'Disgust_f1-score': 0.021052631578947368,
  'Fear_precision': 0.0,
  'Fear_recall': (0.0,),
  'Fear_f1-score': 0.0,
  'Happiness_precision': 0.22348484848484848,
  'Happiness_recall': (0.3155080213903743,),
  'Happiness_f1-score': 0.26164079822616404,
  'Neutrality_precision': 0.235202492211838,
  'Neutrality_recall': (0.9041916167664671,),
  'Neutrality_f1-score': 0.37330037082818296,
  'Sadness_precision': 0.0,
  'Sadness_recall': (0.0,),
  'Sadness_f1-score': 0.0,
  'Surprise_precision': 0.43243243243243246,
  'Surprise_recall

In [39]:
exp_clf_with_feature_selected(adaBoost1, X_train, X_test, y_train_e, y_test_e)

Model Name: AdaBoostClassifier;
Train set shape (24885, 128), num of class 8
              precision    recall  f1-score   support

       Anger       0.51      0.45      0.48       187
    Calmness       0.44      1.00      0.62        16
     Disgust       0.80      0.02      0.04       187
        Fear       0.00      0.00      0.00       187
   Happiness       0.25      0.48      0.33       187
  Neutrality       0.26      0.82      0.39       167
     Sadness       1.00      0.01      0.01       187
    Surprise       0.50      0.69      0.58        62

    accuracy                           0.32      1180
   macro avg       0.47      0.43      0.31      1180
weighted avg       0.48      0.32      0.23      1180

[[ 84   1   0   0  88  13   0   1]
 [  0  16   0   0   0   0   0   0]
 [ 10   2   4   0  56 111   0   4]
 [ 35   1   0   0  68  75   0   8]
 [ 21   3   1   0  90  49   0  23]
 [  2   6   0   0  21 137   0   1]
 [  0   6   0   0  26 148   1   6]
 [ 12   1   0   0   6   0  

({'accuracy': 0.3177966101694915,
  'precision': 0.47089954255447214,
  'recall': 0.4338908649838498,
  'f1-score': 0.3063668790123705,
  'Anger_precision': 0.5121951219512195,
  'Anger_recall': (0.44919786096256686,),
  'Anger_f1-score': 0.47863247863247865,
  'Calmness_precision': 0.4444444444444444,
  'Calmness_recall': (1.0,),
  'Calmness_f1-score': 0.6153846153846153,
  'Disgust_precision': 0.8,
  'Disgust_recall': (0.0213903743315508,),
  'Disgust_f1-score': 0.041666666666666664,
  'Fear_precision': 0.0,
  'Fear_recall': (0.0,),
  'Fear_f1-score': 0.0,
  'Happiness_precision': 0.2535211267605634,
  'Happiness_recall': (0.48128342245989303,),
  'Happiness_f1-score': 0.3321033210332103,
  'Neutrality_precision': 0.2570356472795497,
  'Neutrality_recall': (0.8203592814371258,),
  'Neutrality_f1-score': 0.3914285714285714,
  'Sadness_precision': 1.0,
  'Sadness_recall': (0.0053475935828877,),
  'Sadness_f1-score': 0.010638297872340425,
  'Surprise_precision': 0.5,
  'Surprise_recall'

In [40]:
exp_clf_with_feature_selected(adaBoost2, X_train, X_test, y_train_e, y_test_e)

Model Name: AdaBoostClassifier;
Train set shape (24885, 128), num of class 8
              precision    recall  f1-score   support

       Anger       0.42      0.71      0.52       187
    Calmness       0.52      0.94      0.67        16
     Disgust       0.48      0.11      0.17       187
        Fear       0.40      0.10      0.16       187
   Happiness       0.19      0.13      0.15       187
  Neutrality       0.26      0.79      0.39       167
     Sadness       0.18      0.02      0.04       187
    Surprise       0.48      0.63      0.54        62

    accuracy                           0.33      1180
   macro avg       0.36      0.43      0.33      1180
weighted avg       0.33      0.33      0.26      1180

[[132   0   0   2  28  19   0   6]
 [  0  15   0   0   0   0   1   0]
 [ 41   0  20   6  18  92   6   4]
 [ 40   1   5  19  36  74   6   6]
 [ 69   2   8   9  24  51   1  23]
 [  9   5   2   7   7 132   4   1]
 [  9   6   6   5  11 143   4   3]
 [ 16   0   1   0   5   1  

({'accuracy': 0.326271186440678,
  'precision': 0.3635342071706433,
  'recall': 0.4276403178419769,
  'f1-score': 0.3310677700083077,
  'Anger_precision': 0.4177215189873418,
  'Anger_recall': (0.7058823529411765,),
  'Anger_f1-score': 0.5248508946322067,
  'Calmness_precision': 0.5172413793103449,
  'Calmness_recall': (0.9375,),
  'Calmness_f1-score': 0.6666666666666667,
  'Disgust_precision': 0.47619047619047616,
  'Disgust_recall': (0.10695187165775401,),
  'Disgust_f1-score': 0.17467248908296942,
  'Fear_precision': 0.3958333333333333,
  'Fear_recall': (0.10160427807486631,),
  'Fear_f1-score': 0.16170212765957445,
  'Happiness_precision': 0.18604651162790697,
  'Happiness_recall': (0.12834224598930483,),
  'Happiness_f1-score': 0.1518987341772152,
  'Neutrality_precision': 0.2578125,
  'Neutrality_recall': (0.7904191616766467,),
  'Neutrality_f1-score': 0.38880706921944036,
  'Sadness_precision': 0.18181818181818182,
  'Sadness_recall': (0.0213903743315508,),
  'Sadness_f1-score':

In [41]:
exp_clf_with_feature_selected(adaBoost3, X_train, X_test, y_train_e, y_test_e)

Model Name: AdaBoostClassifier;
Train set shape (24885, 128), num of class 8
              precision    recall  f1-score   support

       Anger       0.36      0.74      0.49       187
    Calmness       0.56      0.88      0.68        16
     Disgust       0.42      0.13      0.20       187
        Fear       0.47      0.14      0.22       187
   Happiness       0.25      0.18      0.21       187
  Neutrality       0.28      0.72      0.41       167
     Sadness       0.49      0.11      0.18       187
    Surprise       0.37      0.29      0.32        62

    accuracy                           0.34      1180
   macro avg       0.40      0.40      0.34      1180
weighted avg       0.38      0.34      0.29      1180

[[139   0   3   2  34   7   0   2]
 [  0  14   0   0   0   2   0   0]
 [ 60   0  25   7  13  80   1   1]
 [ 44   1   5  27  33  63   9   5]
 [ 79   1   9   8  34  35   2  19]
 [ 15   3   5   5   8 121   9   1]
 [ 13   6  10   5   7 122  21   3]
 [ 32   0   2   3   6   0  

({'accuracy': 0.338135593220339,
  'precision': 0.4012817002654004,
  'recall': 0.40067268752989105,
  'f1-score': 0.33994250816425386,
  'Anger_precision': 0.36387434554973824,
  'Anger_recall': (0.7433155080213903,),
  'Anger_f1-score': 0.48857644991212656,
  'Calmness_precision': 0.56,
  'Calmness_recall': (0.875,),
  'Calmness_f1-score': 0.6829268292682927,
  'Disgust_precision': 0.423728813559322,
  'Disgust_recall': (0.13368983957219252,),
  'Disgust_f1-score': 0.2032520325203252,
  'Fear_precision': 0.47368421052631576,
  'Fear_recall': (0.1443850267379679,),
  'Fear_f1-score': 0.22131147540983606,
  'Happiness_precision': 0.2518518518518518,
  'Happiness_recall': (0.18181818181818182,),
  'Happiness_f1-score': 0.21118012422360247,
  'Neutrality_precision': 0.2813953488372093,
  'Neutrality_recall': (0.7245508982035929,),
  'Neutrality_f1-score': 0.40536013400335,
  'Sadness_precision': 0.4883720930232558,
  'Sadness_recall': (0.11229946524064172,),
  'Sadness_f1-score': 0.18260

In [42]:
exp_clf_with_feature_selected(adaBoost4, X_train, X_test, y_train_e, y_test_e)

Model Name: AdaBoostClassifier;
Train set shape (24885, 128), num of class 8
              precision    recall  f1-score   support

       Anger       0.00      0.00      0.00       187
    Calmness       0.00      0.00      0.00        16
     Disgust       0.00      0.00      0.00       187
        Fear       0.00      0.00      0.00       187
   Happiness       0.23      0.51      0.32       187
  Neutrality       0.21      0.96      0.34       167
     Sadness       0.00      0.00      0.00       187
    Surprise       0.00      0.00      0.00        62

    accuracy                           0.22      1180
   macro avg       0.06      0.18      0.08      1180
weighted avg       0.07      0.22      0.10      1180

[[  0   0   0   0 130  57   0   0]
 [  0   0   0   0   0  16   0   0]
 [  0   0   0   0  25 162   0   0]
 [  0   0   0   0  91  96   0   0]
 [  0   0   0   0  96  91   0   0]
 [  0   0   0   0   7 160   0   0]
 [  0   0   0   0  14 173   0   0]
 [  0   0   0   0  49  13  

({'accuracy': 0.21694915254237288,
  'precision': 0.05516788025889968,
  'recall': 0.18393160203656858,
  'f1-score': 0.08284752662637374,
  'Anger_precision': 0.0,
  'Anger_recall': (0.0,),
  'Anger_f1-score': 0.0,
  'Calmness_precision': 0.0,
  'Calmness_recall': (0.0,),
  'Calmness_f1-score': 0.0,
  'Disgust_precision': 0.0,
  'Disgust_recall': (0.0,),
  'Disgust_f1-score': 0.0,
  'Fear_precision': 0.0,
  'Fear_recall': (0.0,),
  'Fear_f1-score': 0.0,
  'Happiness_precision': 0.23300970873786409,
  'Happiness_recall': (0.5133689839572193,),
  'Happiness_f1-score': 0.320534223706177,
  'Neutrality_precision': 0.20833333333333334,
  'Neutrality_recall': (0.9580838323353293,),
  'Neutrality_f1-score': 0.34224598930481287,
  'Sadness_precision': 0.0,
  'Sadness_recall': (0.0,),
  'Sadness_f1-score': 0.0,
  'Surprise_precision': 0.0,
  'Surprise_recall': (0.0,),
  'Surprise_f1-score': 0.0,
  'num_classes': 8,
  'class_names': ['Disgust',
   'Anger',
   'Fear',
   'Sadness',
   'Happiness

In [43]:
exp_clf_with_feature_selected(adaBoost5, X_train, X_test, y_train_e, y_test_e)

Model Name: AdaBoostClassifier;
Train set shape (24885, 128), num of class 8
              precision    recall  f1-score   support

       Anger       0.00      0.00      0.00       187
    Calmness       0.00      0.00      0.00        16
     Disgust       0.00      0.00      0.00       187
        Fear       0.00      0.00      0.00       187
   Happiness       0.23      0.55      0.32       187
  Neutrality       0.22      0.96      0.36       167
     Sadness       0.00      0.00      0.00       187
    Surprise       0.57      0.06      0.12        62

    accuracy                           0.23      1180
   macro avg       0.13      0.20      0.10      1180
weighted avg       0.10      0.23      0.11      1180

[[  0   0   0   0 146  41   0   0]
 [  0   0   0   0   0  16   0   0]
 [  0   0   0   0  32 155   0   0]
 [  0   0   0   0  93  92   0   2]
 [  0   0   0   0 102  85   0   0]
 [  0   0   0   0   6 161   0   0]
 [  0   0   0   0  13 173   0   1]
 [  0   0   0   0  51   7  

({'accuracy': 0.22627118644067798,
  'precision': 0.12777810295397418,
  'recall': 0.19675531634677856,
  'f1-score': 0.09984073897117376,
  'Anger_precision': 0.0,
  'Anger_recall': (0.0,),
  'Anger_f1-score': 0.0,
  'Calmness_precision': 0.0,
  'Calmness_recall': (0.0,),
  'Calmness_f1-score': 0.0,
  'Disgust_precision': 0.0,
  'Disgust_recall': (0.0,),
  'Disgust_f1-score': 0.0,
  'Fear_precision': 0.0,
  'Fear_recall': (0.0,),
  'Fear_f1-score': 0.0,
  'Happiness_precision': 0.23024830699774265,
  'Happiness_recall': (0.5454545454545454,),
  'Happiness_f1-score': 0.32380952380952377,
  'Neutrality_precision': 0.22054794520547946,
  'Neutrality_recall': (0.9640718562874252,),
  'Neutrality_f1-score': 0.35897435897435903,
  'Sadness_precision': 0.0,
  'Sadness_recall': (0.0,),
  'Sadness_f1-score': 0.0,
  'Surprise_precision': 0.5714285714285714,
  'Surprise_recall': (0.06451612903225806,),
  'Surprise_f1-score': 0.11594202898550726,
  'num_classes': 8,
  'class_names': ['Disgust',
 

In [44]:
exp_clf_with_feature_selected(adaBoost6, X_train, X_test, y_train_e, y_test_e)

Model Name: AdaBoostClassifier;
Train set shape (24885, 128), num of class 8
              precision    recall  f1-score   support

       Anger       0.00      0.00      0.00       187
    Calmness       0.33      0.19      0.24        16
     Disgust       0.00      0.00      0.00       187
        Fear       0.00      0.00      0.00       187
   Happiness       0.24      0.66      0.35       187
  Neutrality       0.24      0.92      0.38       167
     Sadness       0.00      0.00      0.00       187
    Surprise       0.53      0.13      0.21        62

    accuracy                           0.24      1180
   macro avg       0.17      0.24      0.15      1180
weighted avg       0.10      0.24      0.12      1180

[[  0   0   0   0 163  23   0   1]
 [  0   3   0   0   0  13   0   0]
 [  0   0   0   0  54 133   0   0]
 [  0   0   0   0  93  91   0   3]
 [  0   0   0   0 123  64   0   0]
 [  0   3   0   0  11 153   0   0]
 [  0   3   0   0  16 165   0   3]
 [  0   0   0   0  51   3  

({'accuracy': 0.24322033898305084,
  'precision': 0.1680725587463402,
  'recall': 0.23630674167879523,
  'f1-score': 0.14713437856482023,
  'Anger_precision': 0.0,
  'Anger_recall': (0.0,),
  'Anger_f1-score': 0.0,
  'Calmness_precision': 0.3333333333333333,
  'Calmness_recall': (0.1875,),
  'Calmness_f1-score': 0.24000000000000005,
  'Disgust_precision': 0.0,
  'Disgust_recall': (0.0,),
  'Disgust_f1-score': 0.0,
  'Fear_precision': 0.0,
  'Fear_recall': (0.0,),
  'Fear_f1-score': 0.0,
  'Happiness_precision': 0.24070450097847357,
  'Happiness_recall': (0.6577540106951871,),
  'Happiness_f1-score': 0.35243553008595985,
  'Neutrality_precision': 0.2372093023255814,
  'Neutrality_recall': (0.9161676646706587,),
  'Neutrality_f1-score': 0.3768472906403941,
  'Sadness_precision': 0.0,
  'Sadness_recall': (0.0,),
  'Sadness_f1-score': 0.0,
  'Surprise_precision': 0.5333333333333333,
  'Surprise_recall': (0.12903225806451613,),
  'Surprise_f1-score': 0.2077922077922078,
  'num_classes': 8,


In [45]:
exp_clf_with_feature_selected(adaBoost7, X_train, X_test, y_train_e, y_test_e)

Model Name: AdaBoostClassifier;
Train set shape (24885, 128), num of class 8
              precision    recall  f1-score   support

       Anger       0.37      0.48      0.42       187
    Calmness       0.44      0.50      0.47        16
     Disgust       0.27      0.09      0.13       187
        Fear       0.45      0.05      0.09       187
   Happiness       0.26      0.44      0.33       187
  Neutrality       0.26      0.73      0.39       167
     Sadness       0.50      0.03      0.05       187
    Surprise       0.45      0.34      0.39        62

    accuracy                           0.30      1180
   macro avg       0.37      0.33      0.28      1180
weighted avg       0.36      0.30      0.24      1180

[[ 90   0   8   3  78   7   0   1]
 [  0   8   0   2   0   6   0   0]
 [ 29   0  16   0  56  82   2   2]
 [ 46   1   3   9  49  72   0   7]
 [ 45   1  13   0  82  35   2   9]
 [  8   3  11   5  14 122   1   3]
 [  4   5   8   1  21 139   5   4]
 [ 23   0   1   0  17   0  

({'accuracy': 0.29915254237288136,
  'precision': 0.3746800699344582,
  'recall': 0.33118281291479484,
  'f1-score': 0.2815683768132501,
  'Anger_precision': 0.3673469387755102,
  'Anger_recall': (0.48128342245989303,),
  'Anger_f1-score': 0.4166666666666667,
  'Calmness_precision': 0.4444444444444444,
  'Calmness_recall': (0.5,),
  'Calmness_f1-score': 0.47058823529411764,
  'Disgust_precision': 0.26666666666666666,
  'Disgust_recall': (0.0855614973262032,),
  'Disgust_f1-score': 0.12955465587044535,
  'Fear_precision': 0.45,
  'Fear_recall': (0.0481283422459893,),
  'Fear_f1-score': 0.08695652173913042,
  'Happiness_precision': 0.2586750788643533,
  'Happiness_recall': (0.4385026737967914,),
  'Happiness_f1-score': 0.3253968253968254,
  'Neutrality_precision': 0.2634989200863931,
  'Neutrality_recall': (0.7305389221556886,),
  'Neutrality_f1-score': 0.3873015873015873,
  'Sadness_precision': 0.5,
  'Sadness_recall': (0.026737967914438502,),
  'Sadness_f1-score': 0.050761421319796954,

In [46]:
exp_clf_with_feature_selected(adaBoost8, X_train, X_test, y_train_e, y_test_e)

Model Name: AdaBoostClassifier;
Train set shape (24885, 128), num of class 8
              precision    recall  f1-score   support

       Anger       0.24      0.93      0.38       187
    Calmness       0.33      1.00      0.50        16
     Disgust       0.25      0.01      0.01       187
        Fear       0.00      0.00      0.00       187
   Happiness       0.00      0.00      0.00       187
  Neutrality       0.15      0.25      0.18       167
     Sadness       0.37      0.12      0.18       187
    Surprise       0.14      0.13      0.14        62

    accuracy                           0.22      1180
   macro avg       0.19      0.30      0.17      1180
weighted avg       0.17      0.22      0.13      1180

[[174   3   0   0   0   3   1   6]
 [  0  16   0   0   0   0   0   0]
 [112   5   1   0   0  50  11   8]
 [116   3   1   0   1  51   5  10]
 [141   5   2   0   0  15  10  14]
 [105   9   0   0   0  41  10   2]
 [ 36   6   0   0   0 114  23   8]
 [ 45   1   0   0   0   6  

({'accuracy': 0.2228813559322034,
  'precision': 0.1852837396408185,
  'recall': 0.30417059618902614,
  'f1-score': 0.17427015428285203,
  'Anger_precision': 0.23868312757201646,
  'Anger_recall': (0.93048128342246,),
  'Anger_f1-score': 0.37991266375545857,
  'Calmness_precision': 0.3333333333333333,
  'Calmness_recall': (1.0,),
  'Calmness_f1-score': 0.5,
  'Disgust_precision': 0.25,
  'Disgust_recall': (0.0053475935828877,),
  'Disgust_f1-score': 0.010471204188481674,
  'Fear_precision': 0.0,
  'Fear_recall': (0.0,),
  'Fear_f1-score': 0.0,
  'Happiness_precision': 0.0,
  'Happiness_recall': (0.0,),
  'Happiness_f1-score': 0.0,
  'Neutrality_precision': 0.14642857142857144,
  'Neutrality_recall': (0.24550898203592814,),
  'Neutrality_f1-score': 0.18344519015659957,
  'Sadness_precision': 0.3709677419354839,
  'Sadness_recall': (0.12299465240641712,),
  'Sadness_f1-score': 0.1847389558232932,
  'Surprise_precision': 0.14285714285714285,
  'Surprise_recall': (0.12903225806451613,),
  

In [47]:
exp_clf_with_feature_selected(adaBoost9, X_train, X_test, y_train_e, y_test_e)

Model Name: AdaBoostClassifier;
Train set shape (24885, 128), num of class 8
              precision    recall  f1-score   support

       Anger       0.00      0.00      0.00       187
    Calmness       0.00      0.00      0.00        16
     Disgust       0.00      0.00      0.00       187
        Fear       0.00      0.00      0.00       187
   Happiness       0.23      0.53      0.33       187
  Neutrality       0.21      0.96      0.35       167
     Sadness       0.00      0.00      0.00       187
    Surprise       1.00      0.02      0.03        62

    accuracy                           0.22      1180
   macro avg       0.18      0.19      0.09      1180
weighted avg       0.12      0.22      0.10      1180

[[  0   0   0   0 139  48   0   0]
 [  0   0   0   0   0  16   0   0]
 [  0   0   0   0  25 162   0   0]
 [  0   0   0   0  92  95   0   0]
 [  0   0   0   0 100  87   0   0]
 [  0   0   0   0   7 160   0   0]
 [  0   0   0   0  14 173   0   0]
 [  0   0   0   0  49  12  

({'accuracy': 0.2211864406779661,
  'precision': 0.18090314797149432,
  'recall': 0.18862152786027048,
  'f1-score': 0.08822954909556796,
  'Anger_precision': 0.0,
  'Anger_recall': (0.0,),
  'Anger_f1-score': 0.0,
  'Calmness_precision': 0.0,
  'Calmness_recall': (0.0,),
  'Calmness_f1-score': 0.0,
  'Disgust_precision': 0.0,
  'Disgust_recall': (0.0,),
  'Disgust_f1-score': 0.0,
  'Fear_precision': 0.0,
  'Fear_recall': (0.0,),
  'Fear_f1-score': 0.0,
  'Happiness_precision': 0.2347417840375587,
  'Happiness_recall': (0.5347593582887701,),
  'Happiness_f1-score': 0.3262642740619902,
  'Neutrality_precision': 0.21248339973439576,
  'Neutrality_recall': (0.9580838323353293,),
  'Neutrality_f1-score': 0.34782608695652173,
  'Sadness_precision': 0.0,
  'Sadness_recall': (0.0,),
  'Sadness_f1-score': 0.0,
  'Surprise_precision': 1.0,
  'Surprise_recall': (0.016129032258064516,),
  'Surprise_f1-score': 0.031746031746031744,
  'num_classes': 8,
  'class_names': ['Disgust',
   'Anger',
   'F

In [48]:
exp_clf_with_feature_selected(adaBoost10, X_train, X_test, y_train_e, y_test_e)

Model Name: AdaBoostClassifier;
Train set shape (24885, 128), num of class 8
              precision    recall  f1-score   support

       Anger       0.00      0.00      0.00       187
    Calmness       0.50      0.12      0.20        16
     Disgust       0.00      0.00      0.00       187
        Fear       0.00      0.00      0.00       187
   Happiness       0.24      0.65      0.35       187
  Neutrality       0.24      0.93      0.38       167
     Sadness       0.00      0.00      0.00       187
    Surprise       0.57      0.13      0.21        62

    accuracy                           0.24      1180
   macro avg       0.19      0.23      0.14      1180
weighted avg       0.11      0.24      0.12      1180

[[  0   0   0   0 164  23   0   0]
 [  0   2   0   0   0  14   0   0]
 [  0   0   0   0  58 129   0   0]
 [  0   0   0   0  94  90   0   3]
 [  0   0   0   0 122  65   0   0]
 [  0   1   0   0  11 155   0   0]
 [  0   1   0   0  17 166   0   3]
 [  0   0   0   0  51   3  

({'accuracy': 0.24322033898305084,
  'precision': 0.1934644297645287,
  'recall': 0.22932279846895826,
  'f1-score': 0.14236132798689513,
  'Anger_precision': 0.0,
  'Anger_recall': (0.0,),
  'Anger_f1-score': 0.0,
  'Calmness_precision': 0.5,
  'Calmness_recall': (0.125,),
  'Calmness_f1-score': 0.2,
  'Disgust_precision': 0.0,
  'Disgust_recall': (0.0,),
  'Disgust_f1-score': 0.0,
  'Fear_precision': 0.0,
  'Fear_recall': (0.0,),
  'Fear_f1-score': 0.0,
  'Happiness_precision': 0.23597678916827852,
  'Happiness_recall': (0.6524064171122995,),
  'Happiness_f1-score': 0.34659090909090906,
  'Neutrality_precision': 0.24031007751937986,
  'Neutrality_recall': (0.9281437125748503,),
  'Neutrality_f1-score': 0.3817733990147783,
  'Sadness_precision': 0.0,
  'Sadness_recall': (0.0,),
  'Sadness_f1-score': 0.0,
  'Surprise_precision': 0.5714285714285714,
  'Surprise_recall': (0.12903225806451613,),
  'Surprise_f1-score': 0.2105263157894737,
  'num_classes': 8,
  'class_names': ['Disgust',
  

In [49]:
exp_clf_with_feature_selected(adaBoost11, X_train, X_test, y_train_e, y_test_e)

Model Name: AdaBoostClassifier;
Train set shape (24885, 128), num of class 8
              precision    recall  f1-score   support

       Anger       0.00      0.00      0.00       187
    Calmness       0.46      0.38      0.41        16
     Disgust       0.00      0.00      0.00       187
        Fear       0.00      0.00      0.00       187
   Happiness       0.23      0.68      0.34       187
  Neutrality       0.26      0.88      0.40       167
     Sadness       0.00      0.00      0.00       187
    Surprise       0.43      0.29      0.35        62

    accuracy                           0.25      1180
   macro avg       0.17      0.28      0.19      1180
weighted avg       0.10      0.25      0.14      1180

[[  0   0   0   0 171  12   0   4]
 [  0   6   0   0   0  10   0   0]
 [  0   0   0   0  84 103   0   0]
 [  0   0   0   0  94  84   0   9]
 [  0   0   0   0 128  52   0   7]
 [  0   3   0   0  17 147   0   0]
 [  0   4   0   0  25 154   0   4]
 [  0   0   1   0  43   0  

({'accuracy': 0.2533898305084746,
  'precision': 0.17242921669078254,
  'recall': 0.2787567600266089,
  'f1-score': 0.18812852284268833,
  'Anger_precision': 0.0,
  'Anger_recall': (0.0,),
  'Anger_f1-score': 0.0,
  'Calmness_precision': 0.46153846153846156,
  'Calmness_recall': (0.375,),
  'Calmness_f1-score': 0.41379310344827586,
  'Disgust_precision': 0.0,
  'Disgust_recall': (0.0,),
  'Disgust_f1-score': 0.0,
  'Fear_precision': 0.0,
  'Fear_recall': (0.0,),
  'Fear_f1-score': 0.0,
  'Happiness_precision': 0.2277580071174377,
  'Happiness_recall': (0.6844919786096256,),
  'Happiness_f1-score': 0.3417890520694259,
  'Neutrality_precision': 0.2615658362989324,
  'Neutrality_recall': (0.8802395209580839,),
  'Neutrality_f1-score': 0.40329218106995884,
  'Sadness_precision': 0.0,
  'Sadness_recall': (0.0,),
  'Sadness_f1-score': 0.0,
  'Surprise_precision': 0.42857142857142855,
  'Surprise_recall': (0.2903225806451613,),
  'Surprise_f1-score': 0.34615384615384615,
  'num_classes': 8,
 

In [50]:
exp_clf_with_feature_selected(adaBoost12, X_train, X_test, y_train_e, y_test_e)

Model Name: AdaBoostClassifier;
Train set shape (24885, 128), num of class 8
              precision    recall  f1-score   support

       Anger       0.33      0.87      0.48       187
    Calmness       0.53      0.56      0.55        16
     Disgust       0.24      0.07      0.11       187
        Fear       0.56      0.11      0.18       187
   Happiness       0.35      0.14      0.20       187
  Neutrality       0.27      0.74      0.39       167
     Sadness       0.28      0.03      0.05       187
    Surprise       0.35      0.11      0.17        62

    accuracy                           0.31      1180
   macro avg       0.36      0.33      0.27      1180
weighted avg       0.34      0.31      0.23      1180

[[162   0   8   0  13   4   0   0]
 [  0   9   1   2   0   1   3   0]
 [ 73   0  14   1   8  87   4   0]
 [ 69   0   9  20  17  64   3   5]
 [107   0  12   4  27  32   1   4]
 [ 30   3   3   2   4 123   1   1]
 [ 12   5   7   6   2 147   5   3]
 [ 42   0   4   1   6   1  

({'accuracy': 0.31101694915254235,
  'precision': 0.36250254281440897,
  'recall': 0.328897688601579,
  'f1-score': 0.2664018333133079,
  'Anger_precision': 0.32727272727272727,
  'Anger_recall': (0.8663101604278075,),
  'Anger_f1-score': 0.4750733137829912,
  'Calmness_precision': 0.5294117647058824,
  'Calmness_recall': (0.5625,),
  'Calmness_f1-score': 0.5454545454545455,
  'Disgust_precision': 0.2413793103448276,
  'Disgust_recall': (0.0748663101604278,),
  'Disgust_f1-score': 0.11428571428571427,
  'Fear_precision': 0.5555555555555556,
  'Fear_recall': (0.10695187165775401,),
  'Fear_f1-score': 0.17937219730941703,
  'Happiness_precision': 0.35064935064935066,
  'Happiness_recall': (0.1443850267379679,),
  'Happiness_f1-score': 0.20454545454545453,
  'Neutrality_precision': 0.2679738562091503,
  'Neutrality_recall': (0.7365269461077845,),
  'Neutrality_f1-score': 0.3929712460063897,
  'Sadness_precision': 0.2777777777777778,
  'Sadness_recall': (0.026737967914438502,),
  'Sadness_

In [51]:
exp_clf_with_feature_selected(adaBoost13, X_train, X_test, y_train_e, y_test_e)

Model Name: AdaBoostClassifier;
Train set shape (24885, 128), num of class 8
              precision    recall  f1-score   support

       Anger       0.29      0.86      0.43       187
    Calmness       0.41      1.00      0.58        16
     Disgust       0.00      0.00      0.00       187
        Fear       0.12      0.01      0.01       187
   Happiness       0.14      0.01      0.01       187
  Neutrality       0.18      0.37      0.25       167
     Sadness       0.11      0.10      0.10       187
    Surprise       0.28      0.37      0.32        62

    accuracy                           0.24      1180
   macro avg       0.19      0.34      0.21      1180
weighted avg       0.15      0.24      0.15      1180

[[160   0   0   1   0  11   8   7]
 [  0  16   0   0   0   0   0   0]
 [ 84   1   0   0   1  59  33   9]
 [ 92   2   0   1   3  49  25  15]
 [112   2   0   3   1  25  25  19]
 [ 48   9   0   0   0  62  45   3]
 [ 26   7   0   1   0 130  18   5]
 [ 28   2   0   2   2   0  

({'accuracy': 0.23813559322033898,
  'precision': 0.19383807725002694,
  'recall': 0.33809900898565126,
  'f1-score': 0.21360274782851624,
  'Anger_precision': 0.2909090909090909,
  'Anger_recall': (0.8556149732620321,),
  'Anger_f1-score': 0.43419267299864317,
  'Calmness_precision': 0.41025641025641024,
  'Calmness_recall': (1.0,),
  'Calmness_f1-score': 0.5818181818181819,
  'Disgust_precision': 0.0,
  'Disgust_recall': (0.0,),
  'Disgust_f1-score': 0.0,
  'Fear_precision': 0.125,
  'Fear_recall': (0.0053475935828877,),
  'Fear_f1-score': 0.010256410256410256,
  'Happiness_precision': 0.14285714285714285,
  'Happiness_recall': (0.0053475935828877,),
  'Happiness_f1-score': 0.010309278350515464,
  'Neutrality_precision': 0.18452380952380953,
  'Neutrality_recall': (0.3712574850299401,),
  'Neutrality_f1-score': 0.24652087475149104,
  'Sadness_precision': 0.11320754716981132,
  'Sadness_recall': (0.0962566844919786,),
  'Sadness_f1-score': 0.10404624277456648,
  'Surprise_precision': 

In [52]:
exp_clf_with_feature_selected(adaBoost14, X_train, X_test, y_train_e, y_test_e)

Model Name: AdaBoostClassifier;
Train set shape (24885, 128), num of class 8
              precision    recall  f1-score   support

       Anger       0.25      0.96      0.40       187
    Calmness       0.52      0.88      0.65        16
     Disgust       0.18      0.27      0.22       187
        Fear       0.28      0.11      0.16       187
   Happiness       0.18      0.02      0.04       187
  Neutrality       0.46      0.11      0.17       167
     Sadness       0.36      0.04      0.08       187
    Surprise       0.27      0.05      0.08        62

    accuracy                           0.25      1180
   macro avg       0.31      0.30      0.23      1180
weighted avg       0.29      0.25      0.18      1180

[[179   0   2   1   4   1   0   0]
 [  0  14   1   0   0   1   0   0]
 [121   0  51   7   4   1   3   0]
 [108   0  47  21   4   1   5   1]
 [152   4  17   4   4   0   1   5]
 [ 69   3  66   8   0  18   3   0]
 [ 31   6  92  29   2  17   8   2]
 [ 48   0   0   5   4   0  

({'accuracy': 0.25254237288135595,
  'precision': 0.314480783211439,
  'recall': 0.3046985800264229,
  'f1-score': 0.22544401353494292,
  'Anger_precision': 0.2528248587570621,
  'Anger_recall': (0.9572192513368984,),
  'Anger_f1-score': 0.39999999999999997,
  'Calmness_precision': 0.5185185185185185,
  'Calmness_recall': (0.875,),
  'Calmness_f1-score': 0.6511627906976744,
  'Disgust_precision': 0.18478260869565216,
  'Disgust_recall': (0.2727272727272727,),
  'Disgust_f1-score': 0.2203023758099352,
  'Fear_precision': 0.28,
  'Fear_recall': (0.11229946524064172,),
  'Fear_f1-score': 0.1603053435114504,
  'Happiness_precision': 0.18181818181818182,
  'Happiness_recall': (0.0213903743315508,),
  'Happiness_f1-score': 0.03827751196172249,
  'Neutrality_precision': 0.46153846153846156,
  'Neutrality_recall': (0.10778443113772455,),
  'Neutrality_f1-score': 0.1747572815533981,
  'Sadness_precision': 0.36363636363636365,
  'Sadness_recall': (0.0427807486631016,),
  'Sadness_f1-score': 0.07

In [53]:
# change y_lable into emo
# exp_clf_with_feature_selected(gb_fast, X_train, X_test, y_train_e, y_test_e)

In [54]:
# check how long on normal gradient boosting
# exp_clf_with_feature_selected(gboost, X_train, X_test, y_train_e, y_test_e)

### Threshold tuning

In [55]:
# Calculate accuracy for the given threshold

probabilities = adaBoost.predict_proba(X_test)

def calc_acc_by_thres(probabilities, threshold, y_test):
    predictions_adj = []
    # Loop through each sample's probabilities
    for probs in probabilities:
        if probs[0] > threshold:
            pred_class = -1
        elif probs[1] > probs[2]:
            pred_class = 0
        else:
            pred_class = 1
        predictions_adj.append(pred_class)
    accuracy = np.mean(predictions_adj == y_test)
    precision, recall, f1score, _ = precision_recall_fscore_support(y_test_s, predictions_adj, average=None)
    return accuracy, min(f1score), np.var(f1score)

best_threshold = None
best_accuracy = 0.0
best_f1score = 0.0
# best_f1s_var = 10

# Define a range of threshold values to try
threshold_range = np.linspace(0.25, 0.75, 100)
for threshold in threshold_range:
    accuracy, min_f1_score, var_f1_score = calc_acc_by_thres(probabilities, threshold, y_test_s)
    # if accuracy > best_accuracy:
    #     best_accuracy = accuracy
    #     best_threshold = threshold
    if min_f1_score > best_f1score:
        best_f1score = min_f1_score
        best_threshold = threshold
        best_accuracy = accuracy
    # if var_f1_score < best_f1s_var:
    #     best_f1score = min_f1_score
    #     best_threshold = threshold
    #     best_accuracy = accuracy
    #     best_f1s_var = var_f1_score
        

print("Best Threshold:", best_threshold)
print("Best Accuracy:", best_accuracy)
print("Best min f1 score:", best_f1score)
# print("Best var f1 score:", best_f1s_var)

Best Threshold: None
Best Accuracy: 0.0
Best min f1 score: 0.0


In [56]:
# if you want to adjust the threshold; Predict probabilities for each class label;
# best threshold and best_threshold x 110% 120% 130% to see trend
probabilities = adaBoost.predict_proba(X_test)

threshold = best_threshold
print('BEST Threshold:', threshold) 
predictions_adj = []
for probs in probabilities:
    if probs[0] > threshold:
        pred_class = -1
    elif probs[1] > probs[2]:
        pred_class = 0
    else:
        pred_class = 1
    predictions_adj.append(pred_class)
print(classification_report(y_test_s, predictions_adj))
print(confusion_matrix(y_test_s, predictions_adj))
print("Accuracy:", np.mean(predictions_adj == y_test_s))

threshold = best_threshold*1.1
print('Threshold-2:', threshold) 
predictions_adj = []
for probs in probabilities:
    if probs[0] > threshold:
        pred_class = -1
    elif probs[1] > probs[2]:
        pred_class = 0
    else:
        pred_class = 1
    predictions_adj.append(pred_class)
print(classification_report(y_test_s, predictions_adj))
print(confusion_matrix(y_test_s, predictions_adj))
print("Accuracy:", np.mean(predictions_adj == y_test_s))

threshold = best_threshold*1.2
print('Threshold-3:', threshold) 
predictions_adj = []
for probs in probabilities:
    if probs[0] > threshold:
        pred_class = -1
    elif probs[1] > probs[2]:
        pred_class = 0
    else:
        pred_class = 1
    predictions_adj.append(pred_class)
print(classification_report(y_test_s, predictions_adj))
print(confusion_matrix(y_test_s, predictions_adj))
print("Accuracy:", np.mean(predictions_adj == y_test_s))

threshold = best_threshold*1.3
print('Threshold-4:', threshold) 
predictions_adj = []
for probs in probabilities:
    if probs[0] > threshold:
        pred_class = -1
    elif probs[1] > probs[2]:
        pred_class = 0
    else:
        pred_class = 1
    predictions_adj.append(pred_class)
print(classification_report(y_test_s, predictions_adj))
print(confusion_matrix(y_test_s, predictions_adj))
print("Accuracy:", np.mean(predictions_adj == y_test_s))

BEST Threshold: None


TypeError: '>' not supported between instances of 'float' and 'NoneType'